**Lectura de Datos**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
archivo_de_paltas = '/content/drive/My Drive/LuisanaPadilla_CODERHOUSE_DataScientist/avocado.csv'
archivo_de_propiedades = '/content/drive/My Drive/LuisanaPadilla_CODERHOUSE_DataScientist/bsas_realstate_on_sale_properati_dataset_2020.csv'
archivo_de_propiedades_gral = '/content/drive/My Drive/LuisanaPadilla_CODERHOUSE_DataScientist/kc_house_data.csv'

In [ ]:
df_paltas = pd.read_csv(archivo_de_paltas)
df_propiedades = pd.read_csv(archivo_de_propiedades)
df_propiedades_gral = pd.read_csv(archivo_de_propiedades_gral)
print(df_propiedades.head())

In [ ]:
df_propiedades.info()

In [ ]:
df_propiedades['rooms'] = df_propiedades['rooms'].astype(int)
df_propiedades['bedrooms'] = df_propiedades['bedrooms'].astype(int)
print(df_propiedades.head())

**Data Wrangling: Limpieza y transformación de datos**

In [ ]:
avg_surface_by_rooms = df_propiedades.groupby('rooms')['surface_total'].mean()
def llenar_surface_total(row):
    if pd.isnull(row['surface_total']):
        return avg_surface_by_rooms[row['rooms']]
    else:
        return row['surface_total']


df_propiedades['surface_total'] = df_propiedades.apply(llenar_surface_total, axis=1)

avg_surfaceC_by_rooms = df_propiedades.groupby('rooms')['surface_covered'].mean()
def llenar_surface_covered(row):
    if pd.isnull(row['surface_covered']):
        return avg_surfaceC_by_rooms[row['rooms']]
    else:
        return row['surface_covered']
df_propiedades['surface_covered'] = df_propiedades.apply(llenar_surface_covered, axis=1)

avg_bathrooms_by_rooms = df_propiedades.groupby('rooms')['bathrooms'].mean()
def llenar_bathroom(row):
    if pd.isnull(row['bathrooms']):
        return avg_bathrooms_by_rooms[row['rooms']]
    else:
        return row['bathrooms']
df_propiedades['bathrooms'] = df_propiedades.apply(llenar_bathroom, axis=1)

df_propiedades.drop(columns=['lat', 'lon'], inplace=True)

df_propiedades = df_propiedades.dropna(subset=['surface_total', 'surface_covered'])

print(df_propiedades.info())

In [ ]:
df_propiedades.head()

In [ ]:
df_propiedades.tail()

In [ ]:
df_paltas.info()

In [ ]:
df_paltas.head()

In [ ]:
df_paltas.tail()

In [ ]:
df_propiedades_gral.info()

In [ ]:
df_propiedades_gral.head()

In [ ]:
df_propiedades_gral.tail()

In [ ]:
property_counts = df_propiedades['property_type'].value_counts()
print(property_counts)

**EDA: Análisis Exploratorio de datos**

In [ ]:
plt.figure(figsize=(10, 6))
property_counts.plot(kind='bar', color='green')
plt.title('Cantidad de Propiedades por Tipo de Propiedad')
plt.xlabel('Tipo de Propiedad')
plt.ylabel('Cantidad de Propiedades')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_propiedades['created_on'] = pd.to_datetime(df_propiedades['created_on'])
filtered_df = df_propiedades[df_propiedades['property_type'].isin(['Departamento', 'Casa', 'PH'])]
grouped_data = filtered_df.groupby(['property_type', pd.Grouper(key='created_on', freq='M')])['price'].mean().unstack(level=0)
plt.figure(figsize=(14, 8))
for property_type in grouped_data.columns:
    plt.plot(grouped_data.index, grouped_data[property_type], marker='o', label=property_type, color='green')
    plt.title('Evolución del Precio de Propiedades (Departamento, Casa, PH)')
    plt.xlabel('Fecha de Creación')
    plt.ylabel('Precio Promedio')
    plt.legend(title='Tipo de Propiedad')
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(plt.FixedFormatter(grouped_data.index.to_period('M').strftime('%Y-%m')))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# prompt: podemos superponer los 3 graficos anteriores en uno solo donde cada línea de tipo de propiedad tiene un color distinto? (preferiblemente tonalidades de verde)

colors = {'Departamento': '#7CFC00', 'Casa': '#00FF00', 'PH': '#008000'}

plt.figure(figsize=(14, 8))
for property_type, color in colors.items():
    plt.plot(grouped_data.index, grouped_data[property_type], marker='o', color=color, label=property_type)

plt.title('Evolución del Precio de Propiedades (Departamento, Casa, PH)')
plt.xlabel('Fecha de Creación')
plt.ylabel('Precio Promedio')
plt.legend(title='Tipo de Propiedad')
plt.gca().xaxis.set_major_formatter(plt.FixedFormatter(grouped_data.index.to_period('M').strftime('%Y-%m')))
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_paltas['Date'] = pd.to_datetime(df_paltas['Date'])
df_promedio = df_paltas.groupby(['type', pd.Grouper(key='Date', freq='M')])['AveragePrice'].mean().reset_index()
plt.figure(figsize=(10, 6))
colores = ['lightgreen', 'green']
for i, (tipo, datos) in enumerate(df_promedio.groupby('type')):
    plt.plot(datos['Date'], datos['AveragePrice'], label=tipo, color=colores[i % len(colores)], marker='o')
    plt.xlabel('Fecha')
    plt.ylabel('Precio Promedio')
    plt.title('Evolución del precio promedio de las paltas por tipo')
    plt.grid(True)
    plt.legend()

In [ ]:
conteo_paltas_type = df_paltas['type'].value_counts()
print(conteo_paltas_type)

In [ ]:
df_paltas['Date'] = pd.to_datetime(df_paltas['Date'])
df_promedio = df_paltas.groupby(pd.Grouper(key='Date', freq='M'))['AveragePrice'].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(df_promedio['Date'], df_promedio['AveragePrice'], label='Promedio General', color='green', marker='o')
plt.xlabel('Fecha')
plt.ylabel('Precio Promedio')
plt.title('Evolución del precio promedio de las paltas')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
df_paltas['Date'] = pd.to_datetime(df_paltas['Date'])
df_promedio_paltas = df_paltas.groupby(pd.Grouper(key='Date', freq='M'))['AveragePrice'].mean().reset_index()

df_propiedades['created_on'] = pd.to_datetime(df_propiedades['created_on'])
filtered_df = df_propiedades[df_propiedades['property_type'] == 'Departamento']
df_promedio_departamentos = filtered_df.groupby(pd.Grouper(key='created_on', freq='M'))['price'].mean().reset_index()

fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:green'
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Precio Promedio de Paltas', color=color)
ax1.plot(df_promedio_paltas['Date'], df_promedio_paltas['AveragePrice'], color=color, marker='o')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Precio Promedio de Departamentos', color=color)
ax2.plot(df_promedio_departamentos['created_on'], df_promedio_departamentos['price'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Comparación de Precios Promedio: Paltas vs Departamentos')
plt.grid(True)
plt.show()


**Preprocesamiento de Datos**

In [ ]:
df_paltas=pd.get_dummies(df_paltas, columns=['type'])
df_paltas

In [ ]:
df_propiedades=pd.get_dummies(df_propiedades, columns=['property_type', 'l2', 'l3', 'l1', 'currency', 'operation_type'])
df_propiedades

In [ ]:
# Al tener problemas con las fechas, he pedido a IA que me ayude a formatear las columnas de fechas de "created on" de la siguiente manera:

# Definir una fecha base
base_date = pd.to_datetime('2000-01-01')

# Calcular el número de días desde la fecha base
df_propiedades['days_from_base_created'] = (df_propiedades['created_on'] - base_date).dt.days

# Eliminar las columnas originales de fechas si no son necesarias
df_propiedades.drop(['start_date', 'end_date', 'created_on'], axis=1, inplace=True)

# Imprimir el DataFrame para verificar
print(df_propiedades.head())


**Feature Selection**

In [ ]:
X = df_propiedades.drop(['price', 'title', 'description'], axis=1)
y = df_propiedades['price']


**Modelo**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Error Cuadrático Medio (MSE): {mse:.2f}')
print(f'Coeficiente de Determinación (R^2): {r2:.2f}')